<a href="https://colab.research.google.com/github/Isaoliveiraaa/Big-Data-facul/blob/main/trabalho_big_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comandos para realização do trabalho da matéria de Big Data com uso da biblioteca PySpark.

## <font color=red>Observação importante:</font>

<font color=yellow>Trabalho realizado com uso da biblioteca pandas não será aceito!</font>

## Upload do arquivo `imdb-reviews-pt-br.csv` para dentro do Google Colab

In [7]:
!wget https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip -O imdb-reviews-pt-br.zip
!unzip imdb-reviews-pt-br.zip
!rm imdb-reviews-pt-br.zip

--2024-10-20 22:18:21--  https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49549692 (47M) [application/zip]
Saving to: ‘imdb-reviews-pt-br.zip’

imdb-reviews-pt-br. 100%[===================>]  47.25M   192MB/s    in 0.2s    

2024-10-20 22:18:22 (192 MB/s) - ‘imdb-reviews-pt-br.zip’ saved [49549692/49549692]

Archive:  imdb-reviews-pt-br.zip
replace imdb-reviews-pt-br.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Instalação manual das dependências para uso do pyspark no Google Colab

In [8]:
!pip install pyspark

## Importar, instanciar e criar a SparkSession

In [9]:
from pyspark.sql import SparkSession

appName = "PySpark Trabalho de Big Data"
master = "local"

spark = SparkSession.builder.appName(appName).master(master).getOrCreate()

## Criar spark dataframe do CSV utilizando o método read.csv do spark

In [10]:
imdb_df = spark.read.csv('imdb-reviews-pt-br.csv',
                         header=True,
                         quote="\"",
                         escape="\"",
                         encoding="UTF-8")

# Questão 1

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave e o "id" como valor do tipo inteiro

In [ ]:
from google.colab import files

# Fazer upload do arquivo (ex: CSV)
uploaded = files.upload()




Saving imdb-reviews-pt-br.csv to imdb-reviews-pt-br (3).csv


In [11]:
# Obter o nome do arquivo a partir do dicionário de uploads
file_name = next(iter(uploaded))  # Pega a primeira chave do dicionário, que é o nome do arquivo

# Carregar o arquivo CSV em um DataFrame do PySpark
df = spark.read.csv(file_name, header=True, inferSchema=True)


In [12]:
df.show(5)


+---+--------------------+--------------------+--------------------+
| id|             text_en|             text_pt|           sentiment|
+---+--------------------+--------------------+--------------------+
|  1|Once again Mr. Co...|Mais uma vez, o S...|                 neg|
|  2|This is an exampl...|Este é um exemplo...|                 neg|
|  3|First of all I ha...|"Primeiro de tudo...| exceto Paxton e ...|
|  4|Not even the Beat...|Nem mesmo os Beat...|                 neg|
|  5|Brass pictures mo...|Filmes de fotos d...|                 neg|
+---+--------------------+--------------------+--------------------+
only showing top 5 rows



In [14]:
# 1. Carregar o arquivo CSV em um DataFrame do PySpark
df = spark.read.csv(file_name, header=True, inferSchema=True)

# 2. Alterar a coluna 'sentiment' para manter apenas 'neg' e 'pos', trocando outros por null
df_corrected = df.withColumn(
    'sentiment',
    F.when(df['sentiment'].isin(['neg', 'pos']), df['sentiment']).otherwise(None)
)

# 3. Excluir as linhas que contêm valores null na coluna 'sentiment'
df_cleaned = df_corrected.dropna(subset=['sentiment'])

# 4. Visualizar o resultado
df_cleaned.show()


+---+--------------------+--------------------+---------+
| id|             text_en|             text_pt|sentiment|
+---+--------------------+--------------------+---------+
|  1|Once again Mr. Co...|Mais uma vez, o S...|      neg|
|  2|This is an exampl...|Este é um exemplo...|      neg|
|  4|Not even the Beat...|Nem mesmo os Beat...|      neg|
|  5|Brass pictures mo...|Filmes de fotos d...|      neg|
|  7|This German horro...|Este filme de ter...|      neg|
| 10|Wealthy horse ran...|Fazendeiros ricos...|      neg|
| 11|"Cage plays a dru...|"Cage interpreta ...|      neg|
| 12|First of all, I w...|Primeiro de tudo,...|      neg|
| 13|So tell me - what...|Então me diga - q...|      neg|
| 14|A big disappointm...|Uma grande decepç...|      neg|
| 15|This film is abso...|Este filme é abso...|      neg|
| 16|Heres a decidedly...|Heres um decidida...|      neg|
| 17|At the bottom end...|Na parte inferior...|      neg|
| 18|Earth has been de...|A terra foi destr...|      neg|
| 19|Many peop

In [16]:

  # Coloque aqui o seu código para retornar a tupla necessária.
df_cleaned = df_corrected.dropna(subset=['sentiment'])

# 2. Transformar o DataFrame em um RDD
rdd = df_cleaned.rdd

# 3. Criar uma função de mapeamento
mapped_rdd = rdd.map(lambda row: (row['sentiment'], int(row['id'])))

# 4. Mostrar o resultado
mapped_df.show()



  # Apague a linha abaixo para iniciar seu código.


+---------+---+
|sentiment| id|
+---------+---+
|      neg|  1|
|      neg|  2|
|      neg|  4|
|      neg|  5|
|      neg|  7|
|      neg| 10|
|      neg| 11|
|      neg| 12|
|      neg| 13|
|      neg| 14|
|      neg| 15|
|      neg| 16|
|      neg| 17|
|      neg| 18|
|      neg| 19|
|      neg| 21|
|      neg| 27|
|      neg| 32|
|      neg| 35|
|      neg| 37|
+---------+---+
only showing top 20 rows



## Cria funções de REDUCE:

- Criar função de reduce para somar os IDs por "sentiment".

In [23]:

# Coloque aqui o seu código para retornar o resultado necessário.
# 1. Carregar o arquivo CSV em um DataFrame do PySpark
df_cleaned = df_corrected.dropna(subset=['sentiment'])

# 2. Transformar o DataFrame em um RDD
rdd = df_cleaned.rdd

# 3. Criar um RDD com "sentiment" como chave e "id" como valor
mapped_rdd = rdd.map(lambda row: (row['sentiment'], int(row['id'])))

# 4. Criar a função de reduce para somar os IDs por "sentiment"
reduced_rdd = mapped_rdd.reduceByKey(lambda a, b: a + b)

result = reduced_rdd.collect()  # Coleta os dados do RDD
for sentiment, total_id in result:  # Itera sobre os resultados
    print(f"Sentiment: {sentiment}, Total ID: {total_id}")


  # Apague a linha abaixo para iniciar seu código.


Sentiment: neg, Total ID: 247015948
Sentiment: pos, Total ID: 444191288


## Aplicação do map/reduce e visualização do resultado

In [24]:
# Coloque aqui a sua linha de código para aplicar o map/reduce no seu
# dataframe spark e realize o collect() ao final para visualizar os dados.

# Obter o nome do arquivo a partir do dicionário de uploads
file_name = next(iter(uploaded))  # Pega a primeira chave do dicionário, que é o nome do arquivo

# Carregar o arquivo CSV em um DataFrame do PySpark
df = spark.read.csv(file_name, header=True, inferSchema=True)

df.show(5)

# 1. Carregar o arquivo CSV em um DataFrame do PySpark
df = spark.read.csv(file_name, header=True, inferSchema=True)

# 2. Alterar a coluna 'sentiment' para manter apenas 'neg' e 'pos', trocando outros por null
df_corrected = df.withColumn(
    'sentiment',
    F.when(df['sentiment'].isin(['neg', 'pos']), df['sentiment']).otherwise(None)
)

# 3. Excluir as linhas que contêm valores null na coluna 'sentiment'
df_cleaned = df_corrected.dropna(subset=['sentiment'])

# 4. Visualizar o resultado
df_cleaned.show()

# Coloque aqui o seu código para retornar a tupla necessária.
df_cleaned = df_corrected.dropna(subset=['sentiment'])

# 2. Transformar o DataFrame em um RDD
rdd = df_cleaned.rdd

# 3. Criar uma função de mapeamento
mapped_rdd = rdd.map(lambda row: (row['sentiment'], int(row['id'])))

# 4. Mostrar o resultado
mapped_df.show()

# Coloque aqui o seu código para retornar o resultado necessário.
# 1. Carregar o arquivo CSV em um DataFrame do PySpark
df_cleaned = df_corrected.dropna(subset=['sentiment'])

# 2. Transformar o DataFrame em um RDD
rdd = df_cleaned.rdd

# 3. Criar um RDD com "sentiment" como chave e "id" como valor
mapped_rdd = rdd.map(lambda row: (row['sentiment'], int(row['id'])))

# 4. Criar a função de reduce para somar os IDs por "sentiment"
reduced_rdd = mapped_rdd.reduceByKey(lambda a, b: a + b)

result = reduced_rdd.collect()  # Coleta os dados do RDD
for sentiment, total_id in result:  # Itera sobre os resultados
    print(f"Sentiment: {sentiment}, Total ID: {total_id}")


+---+--------------------+--------------------+--------------------+
| id|             text_en|             text_pt|           sentiment|
+---+--------------------+--------------------+--------------------+
|  1|Once again Mr. Co...|Mais uma vez, o S...|                 neg|
|  2|This is an exampl...|Este é um exemplo...|                 neg|
|  3|First of all I ha...|"Primeiro de tudo...| exceto Paxton e ...|
|  4|Not even the Beat...|Nem mesmo os Beat...|                 neg|
|  5|Brass pictures mo...|Filmes de fotos d...|                 neg|
+---+--------------------+--------------------+--------------------+
only showing top 5 rows

+---+--------------------+--------------------+---------+
| id|             text_en|             text_pt|sentiment|
+---+--------------------+--------------------+---------+
|  1|Once again Mr. Co...|Mais uma vez, o S...|      neg|
|  2|This is an exampl...|Este é um exemplo...|      neg|
|  4|Not even the Beat...|Nem mesmo os Beat...|      neg|
|  5|B

# Questão 2:

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave e uma tupla com a soma das palavras de cada texto como valor.

In [25]:

  # Coloque aqui o seu código para retornar a tupla necessária.

  # 1. Carregar o arquivo CSV em um DataFrame do PySpark
df_cleaned = df_corrected.dropna(subset=['sentiment'])

# 2. Transformar o DataFrame em um RDD
rdd = df_cleaned.rdd

# 3. Criar uma função de mapeamento para contar as palavras
def map_sentiment_word_count(row):
    # Contar o número de palavras no texto
    word_count = len(row['text'].split())  # Split para contar as palavras
    return (row['sentiment'], word_count)  # Retorna uma tupla

# 4. Aplicar a função de mapeamento
mapped_rdd = rdd.map(map_sentiment_word_count)


  # Apague a linha abaixo para iniciar seu código.


## Cria funções de REDUCE:

- Criar função de reduce para somar o numero de palavras de cada texto português e inglês por "sentiment".

In [26]:

  # Coloque aqui o seu código para retornar o resultado necessário.

  # 1. Carregar o arquivo CSV em um DataFrame do PySpark
df_cleaned = df_corrected.dropna(subset=['sentiment'])

# 2. Transformar o DataFrame em um RDD
rdd = df_cleaned.rdd

# 3. Criar uma função de mapeamento para contar as palavras
def map_sentiment_word_count(row):
    word_count = len(row['text'].split())  # Contar palavras
    return (row['sentiment'], word_count)  # Retornar tupla (sentiment, word_count)

# 4. Aplicar a função de mapeamento
mapped_rdd = rdd.map(map_sentiment_word_count)

# 5. Criar a função de reduce para somar o número de palavras por "sentiment"
reduced_rdd = mapped_rdd.reduceByKey(lambda a, b: a + b)


  # Apague a linha abaixo para iniciar seu código.


## Aplicação do map/reduce e visualização do resultado

1. Aplicar o map/reduce no seu dataframe spark e realizar o collect() ao final
2. Selecionar os dados referentes aos textos negativos para realizar a subtração.
3. Realizar a subtração das contagens de palavras dos textos negativos para obter o resultado final

In [37]:
# Coloque aqui suas linhas de código final

# Criar função de mapeamento para contar o número de palavras por "sentiment"
def map_sentiment_word_count(row):
    # Contar o número de palavras no texto
    word_count = len(row['text'].split())  # Split para contar as palavras
    return (row['sentiment'], word_count)  # Retorna uma tupla (sentiment, word_count)


# Aplicar o reduce para somar o número de palavras por "sentiment"
reduced_rdd = mapped_rdd.reduceByKey(lambda a, b: a + b)


# 1. Carregar o arquivo CSV em um DataFrame do PySpark
df_cleaned = df_corrected.dropna(subset=['sentiment'])

# 2. Transformar o DataFrame em um RDD
rdd = df_cleaned.rdd

# 3. Aplicar a função de mapeamento para contar as palavras
mapped_rdd = rdd.map(map_sentiment_word_count)

# 4. Criar a função de reduce para somar o número de palavras por "sentiment"
reduced_rdd = mapped_rdd.reduceByKey(lambda a, b: a + b)

# 5. Coletar os resultados após o reduce
result = reduced_rdd.collect()

# 6. Filtrar os textos negativos (sentimento "neg")
negative_rdd = reduced_rdd.filter(lambda x: x[0] == "neg")

# 7. Subtrair as contagens de palavras dos textos negativos
negative_word_count = negative_rdd.map(lambda x: x[1]).reduce(lambda a, b: a - b)

# 8. Exibir o resultado final
print(f"Subtração das contagens de palavras dos textos negativos: {negative_word_count}")




Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 36.0 failed 1 times, most recent failure: Lost task 0.0 in stage 36.0 (TID 30) (cbf9dea5eb7c executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/types.py", line 2378, in __getitem__
    idx = self.__fields__.index(item)
ValueError: 'text' is not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/